In [1]:
import torch
import torch.optim as optim

import torch.nn as nn
import torch.nn.functional as F



class CNNPointDetector(nn.Module):
    def __init__(self):
        super(CNNPointDetector, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2)  # Output (x, y) coordinates
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = F.relu(F.max_pool2d(self.conv3(x), 2))
        x = x.view(-1, 64 * 8 * 8)  # Flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize model
model = CNNPointDetector()
# 初始化模型
model = CNNPointDetector()
model.load_state_dict(torch.load('fan_corner_detector_1126.pth'))  # 加载模型参数
model.train()

C:\Users\raykuo7\AppData\Local\Temp\ipykernel_19508\2707019372.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('fan_corner_detector_112

CNNPointDetector(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=2, bias=True)
)

In [2]:
import pandas as pd
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np

class FanDataset(Dataset):
    def __init__(self, annotations_file, img_size=64):
        self.annotations = pd.read_csv(annotations_file)
        self.img_size = img_size
    
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self, idx):
        img_path = self.annotations.iloc[idx, 0]
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        image = cv2.resize(image, (self.img_size, self.img_size)) / 2047.0  # Resize and normalize
        image = image.reshape(1, self.img_size, self.img_size)  # Reshape for CNN input

        # Label: x and y coordinates of the corner
        label = self.annotations.iloc[idx, 1:3].values.astype(np.float32)
        return torch.tensor(image, dtype=torch.float32), torch.tensor(label, dtype=torch.float32)

# 使用自己的标注文件
dataset = FanDataset('selected_points_1126.csv')
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [ ]:
import torch.optim as optim

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 30
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}", 'about',(num_epochs-epoch)/1.5,'min left')

Epoch [1/30], Loss: 35067.9480 about 20.0 min left
Epoch [2/30], Loss: 19434.7501 about 19.333333333333332 min left
Epoch [3/30], Loss: 11264.8167 about 18.666666666666668 min left
Epoch [4/30], Loss: 8270.8713 about 18.0 min left
Epoch [5/30], Loss: 5529.9567 about 17.333333333333332 min left
Epoch [6/30], Loss: 3399.3801 about 16.666666666666668 min left
Epoch [7/30], Loss: 1796.2812 about 16.0 min left
Epoch [8/30], Loss: 1460.7927 about 15.333333333333334 min left
Epoch [9/30], Loss: 865.1829 about 14.666666666666666 min left
Epoch [10/30], Loss: 903.0855 about 14.0 min left
Epoch [11/30], Loss: 634.2906 about 13.333333333333334 min left
Epoch [12/30], Loss: 666.7213 about 12.666666666666666 min left
Epoch [13/30], Loss: 583.2566 about 12.0 min left
Epoch [14/30], Loss: 567.5652 about 11.333333333333334 min left
Epoch [15/30], Loss: 537.0681 about 10.666666666666666 min left
Epoch [16/30], Loss: 547.7424 about 10.0 min left
Epoch [17/30], Loss: 537.6191 about 9.333333333333334 min 

In [ ]:
torch.save(model.state_dict(), './fan_corner_detector_1127.pth')